<a href="https://colab.research.google.com/github/Gayathri-Chevuru/Fatigue-Detection/blob/main/Fatigue_Detection.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
pip install tensorflow


In [81]:
import scipy
import numpy
import tensorflow as tf
from tensorflow import keras
from tensorflow.keras import Sequential, layers, callbacks
from tensorflow.keras.layers import Dense, LSTM, Dropout, GRU, Bidirectional
import pandas as pd
import numpy as np
import math


from sklearn.preprocessing import MinMaxScaler
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import StandardScaler
from sklearn import svm
from sklearn.metrics import accuracy_score
from matplotlib import pyplot as plt
from IPython.core.pylabtools import figsize

from pandas.plotting import register_matplotlib_converters
register_matplotlib_converters()
import seaborn as sns

In [32]:
# Set random seed for reproducibility
tf.random.set_seed(1234)

file = 'nf.csv'
raw_data = pd.read_csv(file)

df = raw_data.copy()
# Explore the first five rows
df.head()
df.describe()
df.shape

(275423, 4)

In [56]:
df = df.astype({"RPE": int})


In [57]:
df.describe()

,time,sensor1_acc,sensor1_jerk,RPE
count,275423.000000,275423.000000,275423.000000,275423.000000
mean,5508.418233,9.957620,70.739021,0.498085
std,3180.279038,1.765477,58.825797,0.499997
min,0.000000,0.096877,-28.735000,0.000000
25%,2754.200000,9.045100,37.137500,0.000000
50%,5508.400000,9.788200,58.930000,0.000000
75%,8262.650000,10.652000,87.445000,1.000000
max,10995.000000,36.860000,1307.400000,1.000000


In [59]:
df['RPE'].value_counts()

0    138239
1    137184
Name: RPE, dtype: int64

**0----> FATIGUE
1----> NON-FATIGUE**


In [60]:
df.groupby('RPE').mean()

,time,sensor1_acc,sensor1_jerk
RPE,,,
0,2764.760000,10.019599,77.029563
1,8273.176299,9.895165,64.400103


In [61]:
#seperating data and labels
X= df.drop(columns='RPE', axis=1)
Y= df['RPE']

In [62]:
print(X)

            time  sensor1_acc  sensor1_jerk
0           0.00       9.7060        0.0000
1           0.04       9.6877        4.9534
2           0.08       9.8547       10.6270
3           0.12      10.2250        5.3160
4           0.16       9.7965       11.4340
...          ...          ...           ...
275418  10995.00       4.5492       28.4580
275419  10995.00       4.5492       28.4580
275420  10995.00       4.5492       28.4580
275421  10995.00       4.5492       28.4580
275422  10995.00       4.5492       28.4580

[275423 rows x 3 columns]


In [63]:
print(Y)

0         0
1         0
2         0
3         0
4         0
         ..
275418    1
275419    1
275420    1
275421    1
275422    1
Name: RPE, Length: 275423, dtype: int64


**Data Standarisation**

In [65]:
scaler = StandardScaler()

In [67]:
scaler.fit(X)

StandardScaler()

In [69]:
standardized_data =scaler.transform(X)

In [70]:
print(standardized_data)

[[-1.73205815 -0.14252262 -1.20251919]
 [-1.73204557 -0.15288811 -1.11831448]
 [-1.732033   -0.05829596 -1.02186682]
 ...
 [ 1.72519193 -3.06343776 -0.71875096]
 [ 1.72519193 -3.06343776 -0.71875096]
 [ 1.72519193 -3.06343776 -0.71875096]]


In [72]:
X=standardized_data
Y=df['RPE']

In [73]:
print(X)
print(Y)

[[-1.73205815 -0.14252262 -1.20251919]
 [-1.73204557 -0.15288811 -1.11831448]
 [-1.732033   -0.05829596 -1.02186682]
 ...
 [ 1.72519193 -3.06343776 -0.71875096]
 [ 1.72519193 -3.06343776 -0.71875096]
 [ 1.72519193 -3.06343776 -0.71875096]]
0         0
1         0
2         0
3         0
4         0
         ..
275418    1
275419    1
275420    1
275421    1
275422    1
Name: RPE, Length: 275423, dtype: int64


**TRAIN TEST SPLIT**

In [76]:
X_train, X_test, Y_train, Y_test = train_test_split(X,Y, test_size=0.2, stratify=Y, random_state=2)

In [77]:
print(X.shape, X_train.shape, X_test.shape)

(275423, 3) (220338, 3) (55085, 3)


**TRAINING THE MODEL**

In [79]:
classifier = svm.SVC(kernel='linear')

In [80]:
#training SVM classifier
classifier.fit(X_train, Y_train)

SVC(kernel='linear')

**MODEL EVALUATION**

ACCURACY SCORE

In [82]:
#accuracy score on the training data
X_train_prediction = classifier.predict(X_train)
training_data_accuracy= accuracy_score(X_train_prediction, Y_train)

In [83]:
print('Accuracy of the training data:', training_data_accuracy)

Accuracy of the training data: 0.9996006136027377


In [84]:
#accuracy score on the test data
X_test_prediction = classifier.predict(X_test)
test_data_accuracy= accuracy_score(X_test_prediction, Y_test)

In [85]:
print('Accuracy of the test data:', test_data_accuracy)

Accuracy of the test data: 0.9995280021784515


**MAKING A PREDICTIVE SYSTEM**

In [87]:
input_data = (10704, 8.9884, 59.513)

#changing the input data to numpy array
input_data_as_numpy_array = np.asarray(input_data)

#reshape the array as we are predicting for one instance
input_data_reshaped = input_data_as_numpy_array.reshape(1,-1)

#standardize the input data
std_data = scaler.transform(input_data_reshaped)
print(std_data)

prediction = classifier.predict(std_data)
print(prediction)

if (prediction[0]==0):
  print('The person is Not Fatigue')

else:
  print('The person is Fatigue')


[[ 1.63369036 -0.54898568 -0.19083535]]
[1]
The person is Fatigue


/usr/local/lib/python3.7/dist-packages/sklearn/base.py:446: UserWarning: X does not have valid feature names, but StandardScaler was fitted with feature names
  "X does not have valid feature names, but"
